In [ ]:
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import random
import numpy as np

import torch
import torch.nn as nn
import snntorch as snn

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data = pd.read_csv("/datasets/faces/labels.csv")
data["file_name"] = data["file_name"].apply(lambda x: f"/datasets/faces/final_files/{x}")

In [ ]:
data

In [ ]:
plt.figure(figsize=(10,4))
sns.distplot(data.real_age, kde=False, bins=100)
plt.title("Distribution of age")
plt.xlabel("Age")
plt.ylabel("Frequency")
plt.show()

In [ ]:
labels = pd.read_csv('/datasets/faces/labels.csv')
train_datagen = ImageDataGenerator(rescale=1./255)
train_gen_flow = train_datagen.flow_from_dataframe(
        dataframe=labels,
        directory='/datasets/faces/final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        seed=12345)

In [ ]:
features, target = next(train_gen_flow)

fig = plt.figure(figsize=(10,10))
for i in range(5):
    fig.add_subplot(4, 4, i+1)
    plt.imshow(features[i])
    plt.xticks([])
    plt.yticks([])
    plt.tight_layout()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
random_state = 12345
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(random_state)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()
])

In [ ]:
train_, test_ = train_test_split(data, test_size=0.33, random_state=random_state)

In [ ]:
len_train, len_test = len(train_), len(test_)

In [ ]:
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx, 0]
        image = Image.open(img_name)
        label = self.data.iloc[idx, 1]

        if self.transform:
            image = self.transform(image).to(device)

        return image, torch.tensor(label, device=device, dtype=torch.float32)

In [ ]:
batch_size = 100

In [ ]:
train = CustomDataset(train_)
test = CustomDataset(test_)
train = DataLoader(train, batch_size=batch_size, shuffle=True)
test = DataLoader(test, batch_size=batch_size, shuffle=True)

In [ ]:
class SimleConv(nn.Module):
    def __init__(self):
        super(SimleConv, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.max1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.max2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(46656, 1)

    def forward(self, x) -> torch.tensor:
        x = self.conv1(x)
        x = self.max1(x)
        x = self.conv2(x)
        x = self.max2(x)
        x = nn.functional.relu(self.fc(x.flatten(1)))

        return(x)

In [ ]:
simple_conv = SimleConv().to(device)
optimazer = torch.optim.Adam(simple_conv.parameters(), lr=1e-3)
citeration = nn.MSELoss().to(device)

In [ ]:
num_epochs = 10

for _ in range(num_epochs):
    simple_conv.train()
    total_loss = 0
    for image, label in train:

        optimazer.zero_grad()
        out = simple_conv(image)

        loss = citeration(out, label)

        loss.backward()
        optimazer.step()

        total_loss += loss.item()

    average_loss = total_loss / len_train
    print(f'Epoch [{_+1}/{num_epochs}], Loss: {average_loss:.4f}')
simple_conv.eval()

In [ ]:
simple_conv_preds = []
simple_conv_labels = []

In [ ]:
for image, label in test:
    out = simple_conv(image)
    simple_conv_labels.append(label.to("cpu").detach().numpy().tolist()[0])
    simple_conv_preds.append(out.to("cpu").detach().numpy().tolist()[0])

In [ ]:
simple_conv_mae = mean_squared_error(simple_conv_labels, simple_conv_preds, squared=False)
print(f"SimpleConv MAE = {simple_conv_mae:0.4f}")

In [ ]:
class SNNConv(nn.Module):
    def __init__(self):
        super(SNNConv, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)
        self.lif1 = snn.Leaky(beta=0.9, learn_beta=True, learn_threshold=True, learn_graded_spikes_factor=True)
        self.max1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3)
        self.lif2 = snn.Leaky(beta=0.9, learn_beta=True, learn_threshold=True, learn_graded_spikes_factor=True)
        self.max2 = nn.MaxPool2d(2)
        self.fc = nn.Linear(46656, 1)

    def forward(self, x):
        mem1 = self.lif1.init_leaky()
        mem2 = self.lif2.init_leaky()

        for i in range(10):
            cur1 = self.conv1(x)
            spk1, mem1 = self.lif1(self.max1(cur1), mem1)
            cur2 = self.conv2(spk1)
            spk2, mem2 = self.lif2(self.max2(cur2), mem2)
            cur3 = self.fc(spk2.flatten(1))
 
        return nn.functional.relu(cur3)

In [ ]:
snn_conv = SNNConv().to(device)
optimazer = torch.optim.Adam(snn_conv.parameters(), lr=1e-3)
citeration = nn.MSELoss().to(device)

In [ ]:
num_epochs = 10

for _ in range(num_epochs):
    snn_conv.train()
    total_loss = 0
    for image, label in train:

        optimazer.zero_grad()
        out = snn_conv(image)
 
        loss = citeration(out, label)
        loss.backward()
        optimazer.step()

        total_loss += loss.item()
        
    average_loss = total_loss / len_train
    print(f'Epoch [{_+1}/{num_epochs}], Loss: {average_loss:.4f}')
snn_conv.eval()

In [ ]:
snn_conv_preds = []
snn_conv_labels = []

In [ ]:
for image, label in test:
    out = snn_conv(image)
    snn_conv_labels.append(label.to("cpu").detach().numpy().tolist()[0])
    snn_conv_preds.append(out.to("cpu").detach().numpy().tolist()[0])

In [ ]:
snn_conv_mae = mean_squared_error(snn_conv_labels, snn_conv_preds, squared=False)
print(f"SNNConv MAE = {snn_conv_mae:0.4f}")

In [ ]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense, AvgPool2D,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet import ResNet50
import numpy as np
import pandas as pd



def load_test(path):
    labels = pd.read_csv(path+'labels.csv') 
    test_datagen = ImageDataGenerator(rescale=1./255,
                                    validation_split=0.25)
    test_datagen_flow = test_datagen.flow_from_dataframe(dataframe=labels,
        directory=path + 'final_files/',
        x_col='file_name',
        y_col='real_age',
        target_size=(224, 224),
        batch_size=32,
        class_mode='raw',
        subset='validation',
        seed=12345) 
    return test_datagen_flow


def load_train(path):
    labels = pd.read_csv(path+'labels.csv') 
    train_datagen = ImageDataGenerator(rescale=1./255,
                                   validation_split=0.25,
                                   horizontal_flip=True,
                                   brightness_range=(0.8, 1.))
 
    train_datagen_flow = train_datagen.flow_from_dataframe(dataframe=labels,
      directory=path + 'final_files/',
      x_col='file_name',
      y_col='real_age',
      target_size=(224, 224),
      batch_size=32,
      class_mode='raw',
      subset='training',
      seed=12345)
    return train_datagen_flow
 
 
def create_model(input_shape):

    backbone = ResNet50(input_shape=input_shape,
                    weights='imagenet', 
                    include_top=False)

    model = Sequential()
    model.add(backbone)
    model.add(GlobalAveragePooling2D()) 
    model.add(Dense(1, activation='relu')) 
    optimizer = Adam(lr=0.0003)
    model.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['mae'])
    
    return model
 
def train_model(model, train_data, test_data, batch_size=None, epochs=10,
               steps_per_epoch=None, validation_steps=None):

    model.fit(train_data, 
              validation_data=test_data,
              batch_size=batch_size, epochs=epochs,
              steps_per_epoch=steps_per_epoch,
              validation_steps=validation_steps,
              verbose=2)
 
    return model